# 1. Import libraries

In [3]:
import cv2
import mediapipe as mp
import numpy as np

/Users/juancarlostapiabaeza/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 2. Set hand and drawing tools

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 3. Display video image

In [5]:
cap = cv2.VideoCapture(1)

cv2.startWindowThread()

while cap.isOpened():

    ret, frame = cap.read()

    cv2.imshow('Frame', frame)

    if cv2.waitKey(10) & 0XFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Necesary to close in mac
cv2.waitKey(1)

2024-02-08 11:14:48.425 Python[1055:11134] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-02-08 11:14:49.873 Python[1055:11134] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


-1

# 4. Display video image with hand recognition

In [7]:
cap = cv2.VideoCapture(1)

cv2.startWindowThread()

with mp_hands.Hands(min_detection_confidence = 0.8, min_tracking_confidence = 0.5) as hands:
    while cap.isOpened():
        # Read frame from video
        ret, frame = cap.read()

        # Flip image
        frame = cv2.flip(frame, 1)
        # Convert image from BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Set flag to false
        image.flags.writeable = False

        # Obtain results
        results = hands.process(image)

        # Set flag to true 
        image.flags.writeable = False

        # Set image back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw hands
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(250, 40, 29), thickness = 3, circle_radius = 6),
                                         mp_drawing.DrawingSpec(color = (230, 35, 40), thickness = 3, circle_radius = 3))
        
        cv2.imshow('Hands Detection', image)


        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()
    
# Necesary to close in mac
cv2.waitKey(1)

I0000 00:00:1707413131.549567       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2


-1

# 4. Function to obtain angle from fingers
<img src = "https://mediapipe.dev/images/mobile/hand_landmarks.png" />

We will obtain the angle from landmarks 4, 0 and 8

In [5]:
list_of_landmarks = [4, 0, 8]

In [6]:
def obtain_angle(list_of_landmarks, hand):
    # Obtain coordinates
    coord_1 = np.array([hand.landmark[list_of_landmarks[0]].x, hand.landmark[list_of_landmarks[0]].y], dtype = 'float') # First coord
    coord_2 = np.array([hand.landmark[list_of_landmarks[1]].x, hand.landmark[list_of_landmarks[1]].y], dtype = 'float') # Second coord
    coord_3 = np.array([hand.landmark[list_of_landmarks[2]].x, hand.landmark[list_of_landmarks[2]].y], dtype = 'float') # Third coord
    
    # Obtain angle in radians
    angle_in_radians = np.abs(np.arctan2(coord_1[1] - coord_2[1], coord_1[0] - coord_2[0])
                            - np.arctan2(coord_3[1] - coord_2[1], coord_3[0] - coord_2[0]))

    # Convert to grads
    angle_in_grads = (angle_in_radians * 180 / np.pi)

    # Obtain only 
    if angle_in_grads > 180:
        angle_in_grads = 360 - angle_in_grads
        
    return angle_in_grads

In [5]:
cap = cv2.VideoCapture(1)

cv2.startWindowThread()

with mp_hands.Hands(min_detection_confidence = 0.8, min_tracking_confidence = 0.5) as hands:
    while cap.isOpened():
        # Read frame from video
        ret, frame = cap.read()

        # Flip image
        frame = cv2.flip(frame, 1)
        # Convert image from BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Set flag to false
        image.flags.writeable = False

        # Obtain results
        results = hands.process(image)

        # Set flag to true 
        image.flags.writeable = False

        # Set image back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw hands
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(250, 40, 29), thickness = 3, circle_radius = 6),
                                         mp_drawing.DrawingSpec(color = (230, 35, 40), thickness = 3, circle_radius = 3))
                # Obtain angle
                angle = obtain_angle(list_of_landmarks, hand)

                # Write angle in image
                cv2.putText(image, str(np.round(angle,2)), 
                            np.multiply(np.array([hand.landmark[list_of_landmarks[1]].x, hand.landmark[list_of_landmarks[1]].y], dtype='float'),[1920.0, 1080.0]).astype(int),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            3,
                            (0, 0, 255)
                            )
                
        cv2.imshow('Hands Detection', image)


        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()
    
# Necesary to close in mac
cv2.waitKey(1)

2024-02-08 11:54:41.409 Python[1468:38767] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1707414882.772559       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-02-08 11:54:42.884 Python[1468:38767] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


-1

# 6. Class to control volume

In [16]:
# Each Block will represent a volume level
class Block:
    level = 0
    width = 120
    height = 40
    color = (0, 0, 255)
    def __init__(self):
        # Set block volume level
        self.blevel = Block.level
        Block.level += 1

        # Bool variable to see if block is active
        self.activated = False

        # Define x_coord and y_coords (to draw in frame)
        self.x_coord = Controler.x_coord
        self.y_coord = Controler.y_coord + Block.height * 6 - (self.blevel + 1) * Block.height

    # Function to obtain block coords in form [(x, y), (x + w, y + h)]
    def rectangle_coords(self):
        coord_1 = (self.x_coord, self.y_coord)
        coord_2 = (self.x_coord + Block.width, self.y_coord + Block.height)

        return [coord_1, coord_2]

    
    def draw_rectangle(self, image):
        cv2.rectangle(image, *self.rectangle_coords(), Block.color, -1)
        print(self.blevel)

# This will be the class in charge of increasing the volume
class Controler:
    width = Block.width
    height = Block.height * 6
    x_coord = 30
    y_coord = 50
    color = (0, 0, 0)

    # Constructor
    def __init__(self):
        self.blocks = []
        for i in range(6):
            self.blocks.append(Block())

        

    # Function to obtain rectangle coords in form [(x, y), (x + w, y + h)]
    def rectangle_coords(self):
        coord_1 = (Controler.x_coord, Controler.y_coord)
        coord_2 = (Controler.x_coord + Controler.width, Controler.y_coord + Controler.height)

        return [coord_1, coord_2]

    # Function to draw Controler rectangle in text
    def draw_controler(self, image):
        cv2.rectangle(image, *self.rectangle_coords(),Controler.color)
        for block in self.blocks:
            if block.activated:
                block.draw_rectangle(image)

    # Check for which blocks to active or deactivate based on angle passed in
    # Angle likely to be in range [0.0, 60.0]
    def state_blocks(self, angle):
        if angle < 5:
            top_block_activated = -1
        else:
            ratio = (60 - 9) / 6
            top_block_activated = int((angle - 9) / ratio)
            
        for num, block in enumerate(self.blocks):
            if num <= top_block_activated:
                block.activated = True
            else:
                block.activated = False
    


In [ ]:
cap = cv2.VideoCapture(1)

cv2.startWindowThread()

controler = Controler()

with mp_hands.Hands(min_detection_confidence = 0.8, min_tracking_confidence = 0.5) as hands:
    while cap.isOpened():
        # Read frame from video
        ret, frame = cap.read()

        # Flip image
        frame = cv2.flip(frame, 1)
        # Convert image from BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Set flag to false
        image.flags.writeable = False

        # Obtain results
        results = hands.process(image)

        # Set flag to true 
        image.flags.writeable = False

        # Set image back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw hands
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(250, 40, 29), thickness = 3, circle_radius = 6),
                                         mp_drawing.DrawingSpec(color = (230, 35, 40), thickness = 3, circle_radius = 3))
                # Obtain angle
                angle = obtain_angle(list_of_landmarks, hand)
                
                controler.state_blocks(angle)

                # Draw Controler
        controler.draw_controler(image)
                
                
        cv2.imshow('Hands Detection', image)


        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()
    
# Necesary to close in mac
cv2.waitKey(1)

I0000 00:00:1707418306.082210       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2


0
0
0
0
0
0
0
0
1
0
1
0
1
2
0
1
2
3
0
1
2
3
0
1
2
3
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
0
1
2
3
0
1
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
2
0
1
2
3
0
1
2
3
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
0
1
2
0
1
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
0
0
0
1
0
1
2
0
1
2
0
1
2
3
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
0
1
2
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
2
0
1
2
0
1
2
3
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
0
1
2
3
0
1
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
2
0
1
2
3
0
1
2
3
0
1
2
3
4
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
